# Homework 6 Projection

**Author**: Chase Coleman

**NYUID**: N10879183

**Date**: 19 March 2016

## Exercise 7:

In this exercise, your job is to project $y$ onto the column space of $X$ in three different ways.

First, use the ordinary expression for the projection. That is

$$Py = X(X'X)^{-1} X'y$$

Next, implement your own version of Gram-Schmidt, based on the algorithm given in the lecture. Use it to convert $X$ to $U$, a matrix with orthonormal columns and the same column space as $X$. Now calculate the projection as 

$$Py = UU'y$$

Finally, use the same expression
$$Py = UU'y$$
but this time obtain $U$ from the QR decomposition routine in either SciPy or Julia.

Test your code with 

$$y = \begin{pmatrix} 1 \\ 3 \\ -3 \end{pmatrix}$$

and

$$X = \begin{pmatrix} 1 & 0 \\ 0 & -6 \\ 2 & 2 \end{pmatrix}$$

You should get the same vector for the projection in each case. Submit this exercise in a separate notebook, with file name `firstname_lastname_projection.ipynb`.

## Solution

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.linalg as la
from mpl_toolkits.mplot3d import axes3d

%matplotlib inline
np.set_printoptions(precision=4, suppress=True)

In [2]:
y = np.array([1.0, 3.0, -3.0])
X = np.array([[1.0, 0.0], [0.0, -6.0], [2.0, 2.0]])

### Compute Expression Directly

In the function below, we will compute the projection as naively as possible including the use of a numerical inverse.

In [3]:
def project_inv(X, y):
    """
    Maps a vector y into column space of X
    
    Parameters
    ----------
    X : Arraylike(Float64, ndim=2)
        Projecting onto span(X)
    y : Arraylike(Float64, ndim=1)
        Want to projecting this vector into span(X)
    
    Returns
    -------
    yhat : Arraylike(Float64, ndim=1)
        Projection of y onto span(X)
        
    Note: Uses a matrix inverse.
    """
    # Projection matrix
    P = X@la.inv(X.T@X)@X.T
    
    # Projection of y into span(X)
    yhat = P@y
    
    return yhat

Below we compute $\hat{y}$ for the direct computation

In [4]:
project_inv(X, y)

array([-0.5652,  3.2609, -2.2174])

### Gram-Schmidt

Here we will compute the projection by using our own implementation of Gram-Schmidt.

In [5]:
def gramschmidt(X):
    """
    Applies Gram-Schmidt to generate a orthonormal matrix 
    with same column space as X
    
    Parameters
    ----------
    X : Arraylike(Float64, ndim=2)
        Will orthonormalize this matrix
        
    Returns
    -------
    U : Arraylike(Float64, ndim=2)
        Orthonormal matrix with same column
        space as X
    """
    # Get sizes of X
    ndim, nvec = np.atleast_2d(X).shape
    
    # Allocate an array to fill with orthonormal matrix
    U = np.empty((ndim, nvec))
    
    # First element of U is a normalized element of X
    U[:, 0] = X[:, 0] / la.norm(X[:, 0])
    
    for i in range(1, nvec):
        # Inputs for step by step projection
        inX = U[:, :i].reshape(ndim, i)
        inY = X[:, i]
        
        # Fill i+1 element by using Gram-Schmidt
        # Note: My = y - Py << Use this below
        MY = inY - inX@inX.T@inY
        U[:, i] = MY / la.norm(MY)
    
    return U


def project_gs(X, y):
    """
    Maps a vector y into column space of X
    
    Parameters
    ----------
    X : Arraylike(Float64, ndim=2)
        Projecting onto span(X)
    y : Arraylike(Float64, ndim=1)
        Want to projecting this vector into span(X)
    
    Returns
    -------
    yhat : Arraylike(Float64, ndim=1)
        Projection of y onto span(X)
        
    Note: Uses a Gram-Schmidt.
    """
    # Orthonormalize X using Gram-Schmidt
    U = gramschmidt(X)
    
    # Do projection
    yhat = U@U.T@y
    
    return yhat


def project_gs_blas(X, y):
    """
    Maps a vector y into column space of X
    
    Parameters
    ----------
    X : Arraylike(Float64, ndim=2)
        Projecting onto span(X)
    y : Arraylike(Float64, ndim=1)
        Want to projecting this vector into span(X)
    
    Returns
    -------
    yhat : Arraylike(Float64, ndim=1)
        Projection of y onto span(X)
        
    Note: Uses a Gram-Schmidt from scipy to give it
    a fair chance at timings below
    """
    U = la.orth(X)
    yhat = U@U.T@y
    return yhat


Computing the projection via Gram-Schmidt gives us the same thing

In [6]:
project_gs(X, y)

array([-0.5652,  3.2609, -2.2174])

### QR Decomposition

We can also compute this projection by using the orthonormality of the $Q$ matrix in the QR decomposition.

In [7]:
def project_qr(X, y):
    """
    Maps a vector y into column space of X
    
    Parameters
    ----------
    X : Arraylike(Float64, ndim=2)
        Projecting onto span(X)
    y : Arraylike(Float64, ndim=1)
        Want to projecting this vector into span(X)
    
    Returns
    -------
    yhat : Arraylike(Float64, ndim=1)
        Projection of y onto span(X)
        
    Note: `scipy.linalg.qr` to get the QR decomp and
    need the `mode=economic` decomposition to get
    the right matrices
    """
    # QR decomposition
    Q, R = la.qr(X, mode='economic')
    
    # Get projection of y onto X
    yhat = Q@Q.T@y
    
    return yhat
    

Computing the projection via QR decomposition also matches our previous two methods.

In [8]:
project_qr(X, y)

array([-0.5652,  3.2609, -2.2174])

### Timing

Below we can run a small horse race among the methods. To make the operations a bit less trivial, we can make the $X$ matrix and $y$ vector a bit bigger.

In [9]:
X = np.random.randn(250, 100)
y = np.random.randn(250)

In [10]:
%timeit project_inv(X, y)

The slowest run took 4.99 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 640 µs per loop


In [11]:
%timeit project_gs_blas(X, y)

The slowest run took 35.99 times longer than the fastest. This could mean that an intermediate result is being cached.
100 loops, best of 3: 3.55 ms per loop


In [12]:
%timeit project_qr(X, y)

1000 loops, best of 3: 1.17 ms per loop


It turns out the inverse method is the fastest, though because of its need to take an inverse it is the least numerically stable.